# Find the Tents

_Combinatorial Optimization course, FEE CTU in Prague. Created by [Industrial Informatics Department](http://industrialinformatics.fel.cvut.cz)._

The problem was taken from https://www.brainbashers.com/tents.asp ; there, you can try to solve some examples manually.

## Task

Find all of the hidden tents in the forest grid.

You know that:

- Each tent is attached to one tree (so there are as many tents as there are trees).
- A tent can only be found horizontally or vertically adjacent to a tree.
- Tents are never adjacent to each other, neither vertically, horizontally, nor diagonally.
- A tree might be next to two tents but is only connected to one.

You are also given two vectors indicating how many tents are in each respective row or column of the forest grid.


## Input

You are given a positive integer $n \geq 2$, representing the size of the forest grid (assume it is a square of size $(n \times n$). You are also given vectors $\mathbf r = (r_1, \dots, r_n)$ and $\mathbf c = (c_1, \dots, c_n)$ representing the numbers of the tents in the rows and columns of the forest grid. Finally, you are given a list of coordinates of the trees $((x_1, y_1), \dots, (x_k, y_k))$.

In [68]:
# 2x2 - Extra small (for debugging)
n1 = 3
r1 = (1, 1, 0)
c1 = (1, 0, 1)
trees1 = [(1,1), (3,2)]


In [69]:
# 8x8 - Medium
n2 = 8
r2 = (3, 1, 1, 2, 0, 2, 0, 3)
c2 = (2, 1, 2, 2 ,1, 1 ,2 ,1)
trees2 = [(2, 1), (5, 1), (6, 1),
         (1, 2),
         (3, 3),
         (3, 4), (6, 4),
         (4, 5), (6, 5),
         (8, 7),
         (2, 8), (4, 8)]

In [70]:
# Weekly special
n3 = 20
r3 = (7, 2, 3, 4, 3, 5, 4, 4, 4, 4, 3, 6, 3, 6, 2, 3, 6, 3, 3, 5)
c3 = (6, 4, 3, 5, 4, 4, 4, 3, 5, 3, 4, 3, 4, 4, 6, 3 ,4, 3, 6, 2)
trees3 = [(3, 1), (4, 1), (8, 1), (13, 1), (15, 1),
         (1, 2), (9, 2), (18, 2), (19, 2),
         (5, 3), (12, 3), (15, 3),
         (2, 4), (4, 4), (9, 4), (17, 4),
         (6, 5), (10, 5), (13, 5), (17, 5), (20, 5),
         (1, 6), (7, 6), (10, 6), (12, 6), (16, 6),
         (20, 7),
         (1, 8), (4, 8), (5, 8), (11, 8), (13, 8), (14, 8), (19, 8),
         (4, 9), (6, 9), (9, 9), (15, 9), (17, 9),
         (8, 10), (17, 10), (19, 10),
         (12, 11),
         (5, 12), (7, 12), (14, 12), (16, 12),
         (1, 13), (2, 13), (6, 13), (19, 13),
         (11, 14), (14, 14), (20, 14),
         (3, 15), (5, 15), (6, 15), (8, 15), (13, 15), (20, 15),
         (2, 16), (3, 16), (10, 16),
         (8, 17), (11, 17), (14, 17), (15, 17),
         (2, 18), (6, 18), (9, 18), (12, 18), (13, 18), (18, 18),
         (2, 19), (7, 19), (15, 19), (17, 19), (20, 19),
         (5, 20), (10, 20)]

## Output

You should find the coordinates $(x_i, y_i), i \in \{1,\dots,k\}$, of the individual tents.

## Model

In [71]:
from gurobipy import *
from itertools import product as cartesian

In [72]:
def optimize(n, r, c, trees):
    m = Model()
    # n+2 -> extend the board such as we don't need check borders
    # this is really nice hack, disable all variables with uper bound 0 and
    # then allow them only in tree neighborhood -> we don't need second matrix
    X = m.addVars(n+2, n+2, vtype=GRB.BINARY, ub=0)
    
    # set sums per rows, iterate only through valid board (not extended)
    for i, val in enumerate(r, start=1):
        m.addConstr(sum(X[j, i] for j in range(1, n+1)) == val)
    # sums per columns
    for j, val in enumerate(c, start=1):
        m.addConstr(sum(X[j, i] for i in range(1, n+1)) == val)
    
    # no other tents near one tent
    tents_neighboorhood = {(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)}
    # sum max of neighberhood + 1
    M = 9
    for i in range(1, n+1):
        for j in range(1, n+1):
            # if there's a tent, there can't be another one around it
            m.addConstr(M * (1 - X[i, j]) >= sum(X[i+ii, j+jj] for ii, jj in tents_neighboorhood))
    
    # as we extended board to n+2, we need to know indicies that are ouside of the board
    outer_frame = set(cartesian(range(n+2), range(n+2))) - set(cartesian(range(1,n+1), range(1,n+1)))
    # tents can be only in these incidies
    allowed_tent_indicies = {(1,0), (-1,0), (0,1), (0,-1)}
    for i, j in trees:
        allowed_neighberhood = [(i+ii, j+jj) for (ii,jj) in allowed_tent_indicies]
        # allow to place tent only if there's tree
        for idx in allowed_neighberhood:
            # if index is inside the board, allow to place tent
            if idx not in outer_frame: 
                # thanks Prokop
                X[idx].ub = 1
            
        # there must be at least one tent in the neighberhood
        m.addConstr(sum(X[idx] for idx in allowed_neighberhood) >= 1)
            
    m.optimize()
    
    return [coords for coords, var in X.items() if var.x > 0]

 ##  Visualization

In [73]:
import matplotlib.pyplot as plt
import numpy as np

def visualize(n, trees, tents, r, c):
    grid = [["." for _ in range(n+2)] for _ in range(n+2)]
    
    for t_x, t_y in tents:
        grid[t_y][t_x] = "X"
    
    for t_x, t_y in trees:
        grid[t_y][t_x] = "T"

    print("  ", end="")
    for c_cur in c:
        print(c_cur, end=" ")
    print()
    
    for y in range(1, n+1):
        print(r[y-1], end=" ")
        for x in range(1, n+1):
            print(grid[y][x], end=" ")
            
        print()

In [74]:
tents1 = optimize(n1, r1, c1, trees1)
visualize(n1, trees1, tents1, r1, c1)

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 17 rows, 25 columns and 107 nonzeros
Model fingerprint: 0xd2371660
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 17 rows and 25 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
  1 0 1 
1 T . X 
1 X . T 
0 . . . 


In [75]:
tents2 = optimize(n2, r2, c2, trees2)
visualize(n2, trees2, tents2, r2, c2)

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 92 rows, 100 columns and 752 nonzeros
Model fingerprint: 0x134b16b8
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
  2 1 2 2 1 1 2 1 
3 X T X . T T X . 
1 T . . . X . . . 
1 . X T . . . . . 
2 . . T X . T X . 
0 . . . T . T . . 
2 . . . X . X . . 
0 . . . . . . . T 
3 X T X T . . . X 


In [76]:
tents3 = optimize(n3, r3, c3, trees3)
visualize(n3, trees3, tents3, r3, c3)

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 520 rows, 484 columns and 4720 nonzeros
Model fingerprint: 0xcc059f6b
Variable types: 0 continuous, 484 integer (484 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 198 rows and 283 columns
Presolve time: 0.01s
Presolved: 322 rows, 201 columns, 1586 nonzeros
Variable types: 0 continuous, 201 integer (201 binary)

Root relaxation: objective 0.000000e+00, 600 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   97          -    0.00000      -     -    0s
H    0     0                       0.0000000    0.00000  0.00%     -    0s
     0     0          -    0         0.00000    0.00000  0.00%     -    0s

Cutting planes:
  Gomor